In [1]:
from filtros import *
from functions import eta_subciclo
from functions import eta_subciclo_nominal
from fit_voc import *
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [6]:
def unnml_factors(df,new_cycles,new_EOL,factors_column,cycles_column):
    new_eta0 = new_EOL**(1/new_cycles)
    sr_numeric_0 = 100
    unnoml_adapted_factors = []
    unnoml_adapted_cycles = []
    for sr_k, factor in zip(df.SR, df.Factors_nml_adap):
        # obtenemos el sr de cada caso
        sr_range = [float(x) for x in sr_k.split("-")]
        sr_numeric = sr_range[0] - sr_range[1]  # SR
        # normlizamos el factor
        eta_k = factor*new_eta0
        unnml_eta_k = eta_k**(sr_numeric/sr_numeric_0)
        unnml_factor = unnml_eta_k/new_eta0
        eq_cycle = 1/(log_a(new_EOL,unnml_eta_k))
        # actualizamos los nuevos valores
        unnoml_adapted_cycles.append(eq_cycle)
        unnoml_adapted_factors.append(unnml_factor)

    factors_column = factors_column + "_unnml"
    cycles_column = cycles_column + "_unnml"
    df[factors_column] = unnoml_adapted_factors
    df[cycles_column] = unnoml_adapted_cycles
    return df

### Tenemos la tabla original de aramis

In [3]:
SR =         ["100-0    ",
              "100-25   ",
              "75-0     ",
              "100-50   ",
              "75-25    ",
              "50-0     ",
              "100-75   ",
              "75-50    ",
              "62.5-37.5",
              "50-25    ",
              "25-0     ",]
og08_factors = [1.00000000,
                1.00000266,
                1.00001860,
                0.99999203,
                1.00001521,
                1.00002874,
                1.00002146,
                1.00000881,
                1.00000620,
                1.00003347,
                1.00004184]
og_data = list(zip(SR,og08_factors))
og_headers = ["SR","Factors"]
df = pd.DataFrame(og_data,columns=og_headers)

# agregamos los subciclos que induce cada factor
cycles_0 = 4000
EOL_0 = 0.8
eta_0 = EOL_0**(1/cycles_0)
subc_k = []

for factor in df.Factors.values:
    eta_k = factor*eta_0
    sub_k = 1/(log_a(EOL_0,eta_k))
    subc_k.append(sub_k)
df["Subciclos"] = subc_k

df

,SR,Factors,Subciclos
0,100-0,1.000000,4000.000000
1,100-25,1.000003,4200.278726
2,75-0,1.000019,6000.731093
3,100-50,0.999992,3499.966007
4,75-25,1.000015,5499.396985
5,50-0,1.000029,8250.427770
6,100-75,1.000021,6500.692875
7,75-50,1.000009,4750.168167
8,62.5-37.5,1.000006,4500.140556
9,50-25,1.000033,9999.062214


In [7]:
new_cycles = 500
new_EOL = 0.7 # asumimos que el EOL es el mismo
df = nml_factors(df,"Factors","Subciclos",eta_0,EOL_0)
df = adap_factors(df,new_cycles,cycles_0,new_EOL,"Factors_nml","Subciclos_nml")
df = unnml_factors(df,new_cycles,new_EOL,"Factors_nml_adap","Subciclos_nml_adap")
df


,SR,Factors,Subciclos,Factors_nml,Subciclos_nml,Factors_nml_adap,Subciclos_nml_adap,Factors_nml_adap_unnml,Subciclos_nml_adap_unnml
0,100-0,1.000000,4000.000000,1.000000,4000.000000,1.000000,500.000000,1.000000,500.000000
1,100-25,1.000003,4200.278726,0.999985,3150.209045,0.999880,427.801699,1.000088,570.402266
2,75-0,1.000019,6000.731093,1.000006,4500.548320,1.000050,537.392446,1.000216,716.523261
3,100-50,0.999992,3499.966007,0.999928,1749.983004,0.999426,277.102798,1.000070,554.205595
4,75-25,1.000015,5499.396985,0.999975,2749.698493,0.999797,389.264630,1.000255,778.529260
5,50-0,1.000029,8250.427770,1.000002,4125.213885,1.000014,509.678637,1.000364,1019.357273
6,100-75,1.000021,6500.692875,0.999918,1625.173219,0.999348,261.204921,1.000372,1044.819682
7,75-50,1.000009,4750.168167,0.999868,1187.542042,0.998944,201.477980,1.000271,805.911919
8,62.5-37.5,1.000006,4500.140556,0.999857,1125.035139,0.998860,192.400837,1.000250,769.603348
9,50-25,1.000033,9999.062214,0.999967,2499.765554,0.999732,363.512943,1.000468,1454.051770
